In [1]:
import numpy as np
import pandas as pd

In [2]:
cd Dropbox/Portfolio/DataScience-Portfolio/KDD-1998

/Users/Capgemini/Dropbox/Portfolio/DataScience-Portfolio/KDD-1998


## Data Loading and Pre-Processing

The data sets are in comma delimited format. The learning dataset, cup98LRN.txt, contains **95412** records and **481** fields. The first row of the data set contains the field names.

In [3]:
# load data, error_bad_lines ignores/skips lines with unusual number of fields, otherwise raises error
kdd = pd.read_csv("cup98lrn.txt", sep = ',', error_bad_lines=False)

b'Skipping line 579: expected 481 fields, saw 961\nSkipping line 1120: expected 481 fields, saw 961\n'
b'Skipping line 2250: expected 481 fields, saw 961\nSkipping line 3325: expected 481 fields, saw 961\n'
b'Skipping line 5556: expected 481 fields, saw 961\nSkipping line 5764: expected 481 fields, saw 961\n'
b'Skipping line 6611: expected 481 fields, saw 961\nSkipping line 7220: expected 481 fields, saw 961\nSkipping line 7877: expected 481 fields, saw 961\n'
b'Skipping line 9272: expected 481 fields, saw 961\nSkipping line 9787: expected 481 fields, saw 961\n'
b'Skipping line 11962: expected 481 fields, saw 961\nSkipping line 12195: expected 481 fields, saw 961\n'
b'Skipping line 12379: expected 481 fields, saw 961\nSkipping line 12560: expected 481 fields, saw 1441\nSkipping line 14103: expected 481 fields, saw 961\n'
b'Skipping line 17083: expected 481 fields, saw 961\n'
b'Skipping line 18634: expected 481 fields, saw 961\nSkipping line 18698: expected 481 fields, saw 961\nSkipping

In [4]:
# check I get expected number of rows and columns
# there are a few less rows than expected because some were skipped due to extra unexpected fields in txt
kdd.shape

(95149, 481)

In [5]:
# check column names are the right ones
kdd.columns

Index(['ODATEDW', 'OSOURCE', 'TCODE', 'STATE', 'ZIP', 'MAILCODE', 'PVASTATE',
       'DOB', 'NOEXCH', 'RECINHSE',
       ...
       'TARGET_D', 'HPHONE_D', 'RFA_2R', 'RFA_2F', 'RFA_2A', 'MDMAUD_R',
       'MDMAUD_F', 'MDMAUD_A', 'CLUSTER2', 'GEOCODE2'],
      dtype='object', length=481)

In [6]:
# quick look at the data
kdd.head()

,ODATEDW,OSOURCE,TCODE,STATE,ZIP,MAILCODE,PVASTATE,DOB,NOEXCH,RECINHSE,...,TARGET_D,HPHONE_D,RFA_2R,RFA_2F,RFA_2A,MDMAUD_R,MDMAUD_F,MDMAUD_A,CLUSTER2,GEOCODE2
0,8901,GRI,0,IL,61081,,,3712,0,,...,0,0,L,4,E,X,X,X,39,C
1,9401,BOA,1,CA,91326,,,5202,0,,...,0,0,L,2,G,X,X,X,1,A
2,9001,AMH,1,NC,27017,,,0,0,,...,0,1,L,4,E,X,X,X,60,C
3,8701,BRY,0,CA,95953,,,2801,0,,...,0,1,L,4,E,X,X,X,41,C
4,8601,,0,FL,33176,,,2001,0,X,...,0,1,L,2,F,X,X,X,26,A


## Data pre-processing following kdd website instructions for columns

In [7]:
# drop CONTROLN as it contains a unique identifier (not useful for predictions).
kdd.drop('CONTROLN', inplace = True, axis = 1)

# drop ZIP too many values, some with format errors. I could correct the format, but here I assume
# that ZIP is not a good classifier.
kdd.drop('ZIP', inplace = True, axis = 1)

# drop OSOURCE too many categorical values, I assume not a good classifier.
kdd.drop('OSOURCE', inplace = True, axis = 1)

# drope TCODE and STATE, too many categorical values
kdd.drop('TCODE', inplace = True, axis = 1)
kdd.drop('STATE', inplace = True, axis = 1)

# drope DOB, we have age as a variable, which is the same concept
kdd.drop('DOB', inplace = True, axis = 1)

In [8]:
# column MAILCODE
# "" indicates address ok, 'B' indicates bad address, will replace by 0 and 1 respectively
kdd.MAILCODE = kdd['MAILCODE'].replace(r'\s+', 0, regex=True)
kdd.MAILCODE = kdd['MAILCODE'].replace('B', 1)

# column PEPSTRFL: Indicates PEP Star RFA Status (whatever that means)
# replace "" with 1 for yes and 'X' for 0 for no, according to KDD dictionary in kdd website
kdd.PEPSTRFL = kdd['PEPSTRFL'].replace(r'\s+', 1, regex=True)
kdd.PEPSTRFL = kdd['PEPSTRFL'].replace('X', 0)  

In [9]:
# DATASRCE is categorical
kdd.DATASRCE = kdd['DATASRCE'].replace(1, 'MetroMail')
kdd.DATASRCE = kdd['DATASRCE'].replace(2, 'Polk')
kdd.DATASRCE = kdd['DATASRCE'].replace(3, 'Both')

In [10]:
# following columns contain numbers that are arbitrary, they refer to a definition, or an arbitrary cluster, but
# are not numerical in escence, so I cast as object

kdd[['SOLP3', 'SOLIH', 'WEALTH2', 'LIFESRC', 'CLUSTER2']] = kdd[['SOLP3', 'SOLIH',
                                                                         'WEALTH2', 'LIFESRC', 
                                                                         'CLUSTER2']].astype(object)


In [11]:
# Blank spaces are missing data: fill blank spaces with NaN
kdd = kdd.replace(r'\s+', np.nan, regex=True)

In [12]:
# Period numbers are missing data: I think they refer to number 99
# Replace value 99 (it appears a lot in most of the columns) with NaN
kdd = kdd.replace(99, np.nan)

In [13]:
# in AGE value 0 indicates missing value
kdd.AGE = kdd.AGE.replace(0, np.nan)

In [14]:
# another quick look to check blanks were filled with NaN
kdd.head()

,ODATEDW,MAILCODE,PVASTATE,NOEXCH,RECINHSE,RECP3,RECPGVG,RECSWEEP,MDMAUD,DOMAIN,...,TARGET_D,HPHONE_D,RFA_2R,RFA_2F,RFA_2A,MDMAUD_R,MDMAUD_F,MDMAUD_A,CLUSTER2,GEOCODE2
0,8901,0,NaN,0,NaN,NaN,NaN,NaN,XXXX,T2,...,0,0,L,4,E,X,X,X,39,C
1,9401,0,NaN,0,NaN,NaN,NaN,NaN,XXXX,S1,...,0,0,L,2,G,X,X,X,1,A
2,9001,0,NaN,0,NaN,NaN,NaN,NaN,XXXX,R2,...,0,1,L,4,E,X,X,X,60,C
3,8701,0,NaN,0,NaN,NaN,NaN,NaN,XXXX,R2,...,0,1,L,4,E,X,X,X,41,C
4,8601,0,NaN,0,X,X,NaN,NaN,XXXX,S2,...,0,1,L,2,F,X,X,X,26,A


In [15]:
# check that 99s were replaced by NaNs
kdd[['POP901', 'POP90C1', 'ETH9', 'AGEC6']].head(6)

,POP901,POP90C1,ETH9,AGEC6
0,992,0,0,11
1,3611,NaN,4,4
2,7001,0,0,10
3,640,0,0,7
4,2520,NaN,0,12
5,18172,81,0,6


## Missing Data

The website indicates that unless a column contains 99.5% NaN, it should be kept for analysis.
For this excercise I will keep only columns that contain less than 70% NaN. I will discard the rest.

In [16]:
# How many NaNs per column should I use as threshold?
.70*95149

66604.3

In [17]:
kdd.isnull().sum().head(10)

ODATEDW         0
MAILCODE        0
PVASTATE    93697
NOEXCH          7
RECINHSE    88465
RECP3       93137
RECPGVG     95035
RECSWEEP    93539
MDMAUD          0
DOMAIN       2300
dtype: int64

In [18]:
# drop columns with 70% NaNs ( we kept 359 from the original 481 columns)
kdd.dropna(axis=1, how='any', thresh=66000, inplace=True)
kdd.shape

(95149, 359)

In [19]:
# make vector with column names
kdd_cols = kdd.columns

In [20]:
# find categorical columns with constant values
for col in kdd_cols:
    if kdd[col].dtype == 'O':
        if kdd[col].describe()['unique']==1:
            print(col)
    

RFA_2R


In [21]:
# verify that same value appears in all rows
kdd['RFA_2R'].describe()

count     95149
unique        1
top           L
freq      95149
Name: RFA_2R, dtype: object

In [22]:
# column contains same value in all rows: drop
kdd.drop('RFA_2R', inplace = True, axis = 1)

In [23]:
# colummns ADATE more often than not, contain same date (same year with a difference of 1-2 months)
# I will consider a date with a difference of a couple of months as same date. To identify them:
# if std is smaller than 0.5 it means that there is only a few days difference => remove

cols_to_remove = []
for num in range(2,23):
    col = 'ADATE' + '_' + str(num)
    if num in [5, 10, 13, 15, 20, 21]: # I eliminated these cols already as full of NaN
        pass
    elif kdd[col].describe()['std'] < 0.5:
        cols_to_remove.append(col)

In [24]:
cols_to_remove

['ADATE_2',
 'ADATE_3',
 'ADATE_6',
 'ADATE_9',
 'ADATE_14',
 'ADATE_16',
 'ADATE_17',
 'ADATE_19']

In [25]:
# to show what I mean by same date and std < 0.5
kdd.ADATE_9.describe()

count    83930.000000
mean      9510.927154
std          0.362431
min       9509.000000
25%       9511.000000
50%       9511.000000
75%       9511.000000
max       9511.000000
Name: ADATE_9, dtype: float64

In [26]:
# remove cols and verify
kdd.drop(cols_to_remove, inplace = True, axis = 1)
kdd.shape

(95149, 350)

## Process NaN:

* For categorical variables, replace NaN with most frequent element
* For numerical variables, replace NaN with median

In [27]:
null_df = pd.DataFrame(kdd.isnull().sum(), columns = ['col'])
null_df.head(10)

,col
ODATEDW,0
MAILCODE,0
NOEXCH,7
MDMAUD,0
DOMAIN,2300
CLUSTER,2300
AGE,23579
HOMEOWNR,22138
INCOME,21198
GENDER,2944


In [28]:
# make vector of columns with NaN
cols_with_nulls = null_df['col'][null_df['col'] > 0].index

In [29]:
for col in cols_with_nulls:
    if kdd[col].dtype == 'O':
        most_frequent_value = kdd[col].describe()['top']
        kdd[col].fillna(most_frequent_value, inplace = True)
    else:
        fill_with_median = kdd[col].median()
        kdd[col].fillna(fill_with_median, inplace = True)

In [30]:
# check that nulls are filled
kdd.isnull().sum().head(20)

ODATEDW     0
MAILCODE    0
NOEXCH      0
MDMAUD      0
DOMAIN      0
CLUSTER     0
AGE         0
HOMEOWNR    0
INCOME      0
GENDER      0
HIT         0
DATASRCE    0
MALEMILI    0
MALEVET     0
VIETVETS    0
WWIIVETS    0
LOCALGOV    0
STATEGOV    0
FEDGOV      0
PEPSTRFL    0
dtype: int64

## Replace date variables with years passed from date to 1997

In [31]:
# I will calculate age respect to 1997
def date_to_years(date):
    yob = str(date)[0:2]
    age = 97 - int(yob)
    return age

In [32]:
kdd['adate4'] = kdd['ADATE_4'].apply(date_to_years)
kdd['adate7'] = kdd['ADATE_7'].apply(date_to_years)
kdd['adate8'] = kdd['ADATE_8'].apply(date_to_years)
kdd['adate11'] = kdd['ADATE_11'].apply(date_to_years)
kdd['adate12'] = kdd['ADATE_12'].apply(date_to_years)
kdd['adate18'] = kdd['ADATE_18'].apply(date_to_years)
kdd['adate22'] = kdd['ADATE_22'].apply(date_to_years)
kdd['odatedw'] = kdd['ODATEDW'].apply(date_to_years)
kdd['maxadate'] = kdd['MAXADATE'].apply(date_to_years)
kdd['minrdate'] = kdd['MINRDATE'].apply(date_to_years)
kdd['maxrdate'] = kdd['MAXRDATE'].apply(date_to_years)
kdd['lastdate'] = kdd['LASTDATE'].apply(date_to_years)
kdd['fistdate'] = kdd['FISTDATE'].apply(date_to_years)
kdd['nextdate'] = kdd['NEXTDATE'].apply(date_to_years)

kdd.adate8.describe()

count    95149.000000
mean         1.067221
std          0.250405
min          1.000000
25%          1.000000
50%          1.000000
75%          1.000000
max          2.000000
Name: adate8, dtype: float64

In [33]:
kdd.drop(['ADATE_4', 'ADATE_7', 'ADATE_8', 'ADATE_11', 'ADATE_12','ADATE_18','ADATE_22',
         'ODATEDW', 'MAXADATE', 'MINRDATE', 'MAXRDATE', 'LASTDATE', 'FISTDATE', 'NEXTDATE'],
         inplace = True, axis = 1)

## Prepare the datasets for regression and classification

* For regression I will try to predict the amount donated. The dataset will include only those people that donated (value 1 in column TARGET_B)
* For classfication, I will use the entire dataset

In [34]:
data_reg = kdd.loc[(kdd.TARGET_B == 1)]
data_reg.drop('TARGET_B', axis = 1, inplace = True)
data_reg.shape

/Users/Capgemini/anaconda/lib/python3.4/site-packages/ipykernel/__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app


(4829, 349)

In [35]:
data_class = kdd.drop('TARGET_D', axis = 1)
data_class.shape

(95149, 349)

In [36]:
data_reg.to_csv('data_reg.csv', header = True, index = False)
data_class.to_csv('data_class.csv', header = True, index = False)

## Make dummy from categorical variables

In [37]:
# How many categorical variables?
cols = kdd.columns
count = 0
cat_vars = []
for col in cols:
    if kdd[col].dtype == 'O':
        count = count + 1
        cat_vars.append(col)
count

27

In [38]:
# make dummies
kdd_dummies = pd.get_dummies(kdd[cat_vars])

# drop categorical vars
kdd.drop(cat_vars, axis = 1, inplace = True)

# concatenate original dataframe with dummies df
kdd = pd.concat([kdd, kdd_dummies], axis = 1)
kdd.shape

(95149, 1915)

In [39]:
# data for regression with dummies
data_reg = kdd.loc[(kdd.TARGET_B == 1)]
data_reg.drop('TARGET_B', axis = 1, inplace = True)

# data for classification
data_class = kdd.drop('TARGET_D', axis = 1)

# save both
data_reg.to_csv('data_reg2.csv', header = True, index = False)
data_class.to_csv('data_class2.csv', header = True, index = False)
kdd.to_csv('kdd.csv', header = True, index = False)

/Users/Capgemini/anaconda/lib/python3.4/site-packages/ipykernel/__main__.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()
